In [ ]:
import argparse
import os
import torch
import torch.backends
from exp.exp_main import Exp_main
from utils.tools import pad_and_convert_to_dict
from utils.tools import save_print
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Subset
import random
import numpy as np
import pandas as pd


fix_seed = 2025
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


parser = argparse.ArgumentParser(description='DFQR')   #  Deep Functional Quantile Regression with Censoring and Structured Interactions

# basic config
parser.add_argument('--model', type=str, default='DFQRwoI',
                    help='model name, options: [DFQR, DFQRwoI]')

# data loader
parser.add_argument('--data', type=str, default='S3_1_c00_01_data0817', help='dataset type')
parser.add_argument('--root_path', type=str, default='./dataset/s3/', help='root path of the data file')
parser.add_argument('--data_path', type=str, default='S3_1_c00_01_data0817.pt', help='data file')
parser.add_argument('--save_path', type=str, default='./save_models/', help='location of saved models')
parser.add_argument('--maxlen_function', type=int, default=None, help='max length of functional data')


# quantile define
parser.add_argument('--num_quantiles', type=int, default=99, help='number of quantiles, e.g. 9 or 99')

# model define
parser.add_argument('--neurons', type=int, nargs='+', default=[64, 128, 64], help='number of neurons in each layer')
parser.add_argument('--dropout', type=float, default=0.01, help='dropout')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--use_norm', type=int, default=1, help='whether to use normalize; True 1 False 0')

# optimization
parser.add_argument('--num_workers', type=int, default=10, help='data loader num workers')
parser.add_argument('--train_epochs', type=int, default=1000, help='train epochs')
parser.add_argument('--batch_size', type=int, default=2048, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=5, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.01, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='MSE', help='loss function')
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')

# GPU
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--gpu_type', type=str, default='cuda', help='gpu type')  # cuda or mps




#args = parser.parse_args()             #save this in .py file
args = parser.parse_args(args=[])       #save this in .ipynb file

setting = '{}_{}_{}_{}_{}'.format(
    args.model,
    args.data,
    args.num_quantiles,
    args.learning_rate,
    args.neurons)
save_print("./log", setting)

if torch.cuda.is_available() and args.use_gpu:
    args.device = torch.device('cuda:{}'.format(args.gpu))
    print('Using GPU')
else:
    if hasattr(torch.backends, "mps"):
        args.device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
    else:
        args.device = torch.device("cpu")
    print('Using cpu or mps')

args.device = torch.device("cuda:0")     # specify device

print('Args in experiment:')
#print_args(args)


df_raw = torch.load(os.path.join(args.root_path, args.data_path))

if isinstance(df_raw, list):
    # If the input is a list, perform padding and convert to dictionary
    df_raw, final_len = pad_and_convert_to_dict(df_raw, target_len=args.maxlen_function)  # target_len is optional
    print(f"Converted list to dictionary format with uniform length, final length is {final_len}")
else:
    print("Data is already in dictionary format, using directly")
    
n = len(df_raw['X_all']) 
I_all = torch.ones(n,1)
train_size = int(0.7*n)
val_size = int(0.1*n)
test_size = n - train_size - val_size

args.p = df_raw['X_all'].shape[-1]
args.q = df_raw['Z_all'].shape[-1]

dataset = torch.utils.data.TensorDataset(I_all[0:n], df_raw['X_all'][0:n], df_raw['t_all'][0:n], df_raw['Z_all'][0:n], df_raw['Y_all'][0:n], df_raw['C_all'][0:n])
train_dataset = Subset(dataset, range(0, train_size))
val_dataset = Subset(dataset, range(train_size, train_size + val_size))
test_dataset = Subset(dataset, range(train_size + val_size, n))

train_loader = DataLoader(train_dataset,batch_size=args.batch_size, shuffle=False)
val_loader = DataLoader(val_dataset,batch_size=args.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset,batch_size=args.batch_size, shuffle=False)
print(f"Train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}, Test set size: {len(test_dataset)}")

Y_pred_all = torch.zeros((n, 1))
for tau in range(args.num_quantiles):
    args.tau = (tau + 1) / (args.num_quantiles + 1)
    
    dataset_pred = torch.utils.data.TensorDataset(Y_pred_all)
    train_pred = Subset(dataset_pred, range(0, train_size))
    val_pred = Subset(dataset_pred, range(train_size, train_size + val_size))
    test_pred = Subset(dataset_pred, range(train_size + val_size, n))

    train_loader_pred = DataLoader(train_pred,batch_size=args.batch_size, shuffle=False)
    val_loader_pred = DataLoader(val_pred,batch_size=args.batch_size, shuffle=False)
    test_loader_pred = DataLoader(test_pred,batch_size=args.batch_size, shuffle=False)

    # setting record of experiments
    exp = Exp_main(args, train_loader, val_loader, test_loader, 
        train_loader_pred, val_loader_pred, test_loader_pred)  # set experiments
    setting = '{}_{}_{}_{}_{}'.format(
        args.model,
        args.data,
        args.num_quantiles,
        args.learning_rate,
        args.neurons)
    
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    print(f"tau={args.tau}")
    exp.train(setting)
    
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)
    
    Y_pred_all = torch.cat([Y_pred_all.cpu(), exp.updateY_pred(setting, I_all, df_raw['X_all'][0:n], df_raw['t_all'][0:n], df_raw['Z_all'][0:n])], dim=1)
    
    if args.gpu_type == 'mps':
        torch.backends.mps.empty_cache()
    elif args.gpu_type == 'cuda':
        torch.cuda.empty_cache()


print(f"Y_pred_all:{Y_pred_all[0,:]}")

Using GPU
Args in experiment:
数据已是字典格式，直接使用
训练集大小:35000,验证集大小:5000,测试集大小:10000
Use GPU: cuda:0


/tmp/ipykernel_46356/3941442525.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  df_raw = torch.load(os.path.join(args.root_path, args.data_path))


>>>>>>>start training : DFQRwoI_S3_1_c00_01_data0817.pt_99_[64, 128, 64]>>>>>>>>>>>>>>>>>>>>>>>>>>
tau=0.01
Epoch: 1 cost time: 1.1380500793457031
Epoch: 1, Steps: 18 | Train Loss: 0.0185974 Vali Loss: 0.0184818 Test Loss: 0.0184273
Validation loss decreased (inf --> 0.018482).  Saving model ...
Updating learning rate to 0.01
Epoch: 2 cost time: 0.866267204284668
Epoch: 2, Steps: 18 | Train Loss: 0.0121809 Vali Loss: 0.0081938 Test Loss: 0.0082107
Validation loss decreased (0.018482 --> 0.008194).  Saving model ...
Updating learning rate to 0.01
Epoch: 3 cost time: 0.8655307292938232
Epoch: 3, Steps: 18 | Train Loss: 0.0089602 Vali Loss: 0.0082886 Test Loss: 0.0083338
EarlyStopping counter: 1 out of 5
Updating learning rate to 0.01
Epoch: 4 cost time: 0.8775832653045654
Epoch: 4, Steps: 18 | Train Loss: 0.0081029 Vali Loss: 0.0080771 Test Loss: 0.0080358
Validation loss decreased (0.008194 --> 0.008077).  Saving model ...
Updating learning rate to 0.01
Epoch: 5 cost time: 0.84423685073

/home/tym/notebooks/tym/jiaqi/dfqr/exp/exp_main.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(best_model_path))
/home/tym/noteb

Test Loss: 0.007813


/home/tym/notebooks/tym/jiaqi/dfqr/exp/exp_main.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(os.path.join('./save_models/', se

Use GPU: cuda:0
>>>>>>>start training : DFQRwoI_S3_1_c00_01_data0817.pt_99_[64, 128, 64]>>>>>>>>>>>>>>>>>>>>>>>>>>
tau=0.02
Epoch: 1 cost time: 0.8994877338409424
Epoch: 1, Steps: 18 | Train Loss: 0.0847050 Vali Loss: 0.0330297 Test Loss: 0.0326299
Validation loss decreased (inf --> 0.033030).  Saving model ...
Updating learning rate to 0.01
Epoch: 2 cost time: 0.8386642932891846
Epoch: 2, Steps: 18 | Train Loss: 0.0311613 Vali Loss: 0.0297863 Test Loss: 0.0296566
Validation loss decreased (0.033030 --> 0.029786).  Saving model ...
Updating learning rate to 0.01
Epoch: 3 cost time: 0.8307771682739258
Epoch: 3, Steps: 18 | Train Loss: 0.0301871 Vali Loss: 0.0291605 Test Loss: 0.0290191
Validation loss decreased (0.029786 --> 0.029161).  Saving model ...
Updating learning rate to 0.01
Epoch: 4 cost time: 0.8459937572479248
Epoch: 4, Steps: 18 | Train Loss: 0.0295277 Vali Loss: 0.0290374 Test Loss: 0.0287841
Validation loss decreased (0.029161 --> 0.029037).  Saving model ...
Updating lea